In [1]:
using JSON

#load OD pair-route incidence
odPairRoute = readall("od_pair_route_incidence_MA_ext.json");
odPairRoute = JSON.parse(odPairRoute);

#load link-route incidence
linkRoute = readall("link_route_incidence_MA_ext.json");
linkRoute = JSON.parse(linkRoute);

#load OD pair labels
odPairLabel = readall("od_pair_label_dict_MA_refined_ext.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readall("od_pair_label_dict__MA_refined_ext.json");
odPairLabel_ = JSON.parse(odPairLabel_);

#load link labels
linkLabel = readall("link_label_dict_MA_ext.json");
linkLabel = JSON.parse(linkLabel);

linkLabel_ = readall("link_label_dict_MA__ext.json");
linkLabel_ = JSON.parse(linkLabel_);

#load node-link incidence
nodeLink = readall("node_link_incidence_MA_ext.json");
nodeLink = JSON.parse(nodeLink);

In [2]:
include("load_network_uni-class.jl")

load_ta_network (generic function with 2 methods)

In [3]:
ta_data = load_ta_network("MA");

In [4]:
capacity = ta_data.capacity;
free_flow_time = ta_data.free_flow_time;

In [5]:
demandsDict = readall("../08_develop_new_OD_demand_estimator_MA_Dijkstra_uni_class/results/demandsDictFixed.json");
demandsDict = JSON.parse(demandsDict);

In [6]:
# demands = demandsDict["$(length(demandsDict)-1)"]

demands = demandsDict

Dict{UTF8String,Any} with 484 entries:
  "(16,1)"  => 0.0
  "(11,20)" => 11.163160759504745
  "(2,8)"   => 0.0
  "(3,22)"  => 0.0
  "(20,1)"  => 0.0
  "(14,5)"  => 0.0
  "(19,5)"  => 0.0
  "(11,1)"  => 0.0
  "(9,6)"   => 0.0
  "(2,17)"  => 0.0
  "(15,20)" => 43.98687765191594
  "(8,12)"  => 1.5818492815661989
  "(22,20)" => 0.0
  "(8,9)"   => 0.0
  "(14,11)" => 0.0
  "(15,22)" => 0.0
  "(13,16)" => 520.6537852451384
  "(19,2)"  => 0.0
  "(2,18)"  => 0.0
  "(11,17)" => 0.0
  "(11,7)"  => 0.0
  "(3,3)"   => 0.0
  "(15,21)" => 13.593190800715998
  "(4,1)"   => 429.3941477957537
  "(16,3)"  => 0.0
  ⋮         => ⋮

In [7]:
odPairLabel_;

In [8]:
demandsVec = zeros(462)

for i = 1:length(demandsVec)
    demandsVec[i] = demands["($(odPairLabel_["$i"][1]),$(odPairLabel_["$i"][2]))"]
end

In [9]:
demandsVec

462-element Array{Float64,1}:
   3.83371
  70.1968 
 284.849  
   0.0    
 110.625  
  78.3549 
   0.0    
   0.0    
   0.0    
   0.0    
 300.292  
   0.0    
 123.378  
   ⋮      
   0.0    
   0.0    
   0.0    
   0.0    
   0.0    
   0.0    
   0.0    
   0.0    
   0.0    
   0.0    
   0.0    
   0.0    

In [10]:
for key=keys(odPairRoute)
    if contains(key, "462-")
        println(key)
    end
end

462-329


In [11]:
linkRoute;

In [12]:
deg = 4
c = 0.5
lam = 100.0

coeffs_dict_Apr_PM_ = readall("../temp_files/coeffs_dict_Apr_PM.json")
coeffs_dict_Apr_PM_ = JSON.parse(coeffs_dict_Apr_PM_)
fcoeffs = coeffs_dict_Apr_PM_["($(deg),$(c),$(lam),1)"]

# fcoeffs = [1, 0, 0, 0, .15]

5-element Array{Any,1}:
 1.0       
 0.0127137 
 0.0213785 
 0.016203  
 0.00541649

In [13]:
using JuMP

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model()

numLinks = 74
numRoute = 4614
numOD = 462

@defVar(m, linkFlow[1:numLinks])
@defVar(m, pathFlow[1:numRoute])

pathFlowSum = Dict()

for i=1:numOD
    pathFlowSum[i] = 0
    for j=1:numRoute
        if "$(i)-$(j)" in keys(odPairRoute)
            pathFlowSum[i] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowSum[i] == demandsVec[i])
end

pathFlowLinkSum = Dict()

for a=1:numLinks
    pathFlowLinkSum[a] = 0
    for j=1:numRoute
        if "$(a)-$(j)" in keys(linkRoute)
            pathFlowLinkSum[a] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowLinkSum[a] == linkFlow[a])
end

for j=1:numRoute
    @addConstraint(m, pathFlow[j] >= 0)
end

# @defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .15*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:numLinks})

@defNLExpr(f, sum{free_flow_time[a] * fcoeffs[1] * linkFlow[a] + 
    free_flow_time[a] * fcoeffs[2] * linkFlow[a]^2 / capacity[a] +
    free_flow_time[a] * fcoeffs[3] * linkFlow[a]^3 / capacity[a]^2 +
    free_flow_time[a] * fcoeffs[4] * linkFlow[a]^4 / capacity[a]^3 +
    free_flow_time[a] * fcoeffs[5] * linkFlow[a]^5 / capacity[a]^4, a = 1:numLinks})

@setNLObjective(m, Min, f)

solve(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2265
Number of nonzeros in inequality constraint Jacobian.:     4614
Number of nonzeros in Lagrangian Hessian.............:       74

Total number of variables............................:     4688
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [14]:
getValue(linkFlow)

74-element Array{Float64,1}:
  790.708     
 1210.67      
  180.822     
 2119.94      
  172.024     
 2446.2       
  930.306     
 2103.15      
  222.189     
  414.305     
   27.8247    
    9.85189   
   -5.45599e-9
    ⋮         
 1003.73      
 1301.38      
 1217.05      
 2249.47      
   -9.49998e-8
   -9.99998e-9
   -9.46083e-8
   -9.99998e-9
  249.471     
  590.207     
  636.375     
  982.092     

In [15]:
getObjectiveValue(m)

9292.76043204409

In [16]:
flows = Dict{(Int64,Int64),Float64}()

for i = 1:length(ta_data.start_node)
    key = (ta_data.start_node[i], ta_data.end_node[i])
    flows[key] = getValue(linkFlow)[i]
end

flows

Dict{(Int64,Int64),Float64} with 74 entries:
  (17,15) => 590.2069500152945
  (3,6)   => 222.18879622462137
  (21,13) => 1189.3375184177705
  (22,16) => -9.999975183757012e-9
  (19,12) => -1.9999950367514024e-8
  (22,14) => -9.460825480779023e-8
  (15,17) => 249.47115426761152
  (10,11) => 140.63469466658356
  (13,12) => 0.0
  (3,1)   => 2119.936467869492
  (16,13) => 1301.3792436141707
  (4,5)   => 986.1882146589488
  (13,16) => 1003.7312437524009
  (17,16) => 982.0918120187399
  (20,13) => 931.4391250501423
  (12,8)  => 410.7344376519842
  (2,4)   => 930.3057033447419
  (16,17) => 636.3746868222452
  (1,2)   => 790.7084701692227
  (14,21) => 1112.8045475462177
  (2,3)   => 172.02440742174915
  (13,19) => -7.999980147005968e-8
  (13,14) => 0.0
  (16,14) => 2249.4746513975324
  (8,12)  => 881.2739006211207
  ⋮       => ⋮

In [17]:
# getValue(linkFlow)

In [18]:
# getObjectiveValue(m)

In [19]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM_ext

In [20]:
flow_user = GLS_Apr_weekday_PM_ext.x_

74x30 Array{Float64,2}:
 1366.35    791.95    771.324  …  1015.01        1624.03     280.272
 1206.76    992.338   991.158     1000.59        1273.13     570.559
 2114.24   1499.68   1200.6       1171.78        1608.83     947.273
 2273.84   1299.3     980.768     1186.21        1959.73     656.986
   58.465   201.97    271.021       86.5748        40.0646   261.282
  139.064   247.675   201.826  …    23.5801       146.69     143.578
 1193.7    1571.81   1353.56       781.132       1109.21    1362.48 
  953.514  1726.49   1642.59       829.701        651.679   1770.47 
 2124.9    1398.79    938.436     1533.92        1629.54    1007.64 
 2365.09   1244.1     649.406     1485.35        2087.07     599.646
  406.406   514.047   567.011  …   469.666        455.737    655.187
  281.997  1677.2    1591.06       204.042        239.965   1660.44 
  204.522   246.959  1020.38         1.2451e-13   268.26    1131.08 
    ⋮                          ⋱                                    
  817.703 

In [21]:
function socialObj(linkFlowVec)
    objVal = sum([free_flow_time[a] * fcoeffs[1] * linkFlowVec[a] + 
        free_flow_time[a] * fcoeffs[2] * linkFlowVec[a]^2 / capacity[a] + 
        free_flow_time[a] * fcoeffs[3] * linkFlowVec[a]^3 / capacity[a]^2 + 
        free_flow_time[a] * fcoeffs[4] * linkFlowVec[a]^4 / capacity[a]^3 + 
        free_flow_time[a] * fcoeffs[5] * linkFlowVec[a]^5 / capacity[a]^4 for a = 1:numLinks])
    return objVal
end

socialObj (generic function with 1 method)

In [22]:
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]
# week_day_Apr_list = [16]

for i in week_day_Apr_list
    println(socialObj(flow_user[:, i])/getObjectiveValue(m))
end

1.8819565822843838
1.984544403618884
1.9960158759543494
2.189690875488253
1.0650868335219192
1.8004286017735354
1.9070264016444674
2.0173427953352814
2.27797179078833
2.027967999213084
1.5445257129920407
1.9339654068103518
1.6411731700137704
1.4485622725650158
1.0647558702712236
2.4029669325054273
1.9367578292137417
1.9450492307188192
1.9128281611623337
1.4623461228678691
1.8780084295832125


In [23]:
length(week_day_Apr_list)

21